In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

# Завантаження даних
import yfinance as yf

# Завантаження даних
data = yf.download(tickers='EURUSD=X', period='1mo', interval='5m')

# Обчислення True Range (TR)
data['High-Low'] = data['High'] - data['Low']
data['High-PrevClose'] = abs(data['High'] - data['Close'].shift(1))
data['Low-PrevClose'] = abs(data['Low'] - data['Close'].shift(1))

data['True Range'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)

# Визначення періоду ATR
atr_period = 200  # зазвичай 14 періодів

# Обчислення ATR як ковзного середнього TR
data['ATR'] = data['True Range'].rolling(window=atr_period).mean()


# Розрахунок щоденної дохідності
data['Daily_Return'] = data['Close']

# Розрахунок волатильності
data['Volatility'] = data['Close'].rolling(window=200).std()


data = data.dropna()

# Підготовка даних
features = data[['Daily_Return', 'Volatility', 'ATR']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Кластеризація
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(scaled_features)


# Візуалізація
plt.figure(figsize=(10, 6))
for cluster in data['Cluster'].unique():
    cluster_data = data[data['Cluster'] == cluster]
    plt.scatter(cluster_data['Daily_Return'], cluster_data['Volatility'], label=f'Cluster {cluster}')
plt.title('Кластеризація ринкових даних (K-Means)')
plt.xlabel('Daily Return (%)')
plt.ylabel('Volatility')
plt.legend()
plt.show()

plt.plot(np.arange(0, len(data['Daily_Return']), 1),scaled_features[:, 0], c='red')
plt.plot(np.arange(0, len(data['Volatility']), 1), scaled_features[:, 1], c='green')
plt.plot(np.arange(0, len(data['ATR']), 1), scaled_features[:, 2], c='blue')

plt.show()


: 

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# 1. Завантаження даних
data = yf.download(tickers='MGC=F', period='1mo', interval='5m')

# 2. Додавання лагових змінних
lags = 50
for lag in range(1, lags + 1):
    data[f'Close_Lag_{lag}'] = data['Close'].shift(lag)

# 3. Видалення NaN
data.dropna(inplace=True)

# 4. Вибір особливостей та цільової змінної
features = [f'Close_Lag_{lag}' for lag in range(1, lags + 1)]
X = data[features]
y = data['Close']

# 5. Розділення на тренувальні та тестові дані
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Виправлення формату y
y_train = y_train.values.ravel()  # Якщо y — DataFrame
y_test = y_test.values.ravel()

# 6. Масштабування даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.1, 1],
    'epsilon': [0.01, 0.1, 0.5]
}
grid_search = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

print("Best parameters:", grid_search.best_params_)
svr_model = grid_search.best_estimator_

svr_model.fit(X_train_scaled, y_train)

# 8. Прогнозування для тестових даних
y_pred = svr_model.predict(X_test_scaled)

# 9. Оцінка моделі
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 10. Візуалізація прогнозів на тестових даних
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual', color='blue')
plt.plot(y_pred, label='Predicted', color='red')
plt.legend()
plt.title('SVR Prediction vs Actual')
plt.show()

# 11. Ітеративне прогнозування майбутніх значень
future_steps = 100  # Кількість кроків у майбутнє
last_known = X_test.iloc[-1].values.reshape(1, -1)  # Останній набір даних
future_predictions = []

for step in range(future_steps):
    # Масштабування останніх даних
    scaled_input = scaler.transform(last_known)

    # Прогноз
    next_prediction = svr_model.predict(scaled_input)[0]
    future_predictions.append(next_prediction)

    # Оновлення останніх даних для наступного кроку
    last_known = np.roll(last_known, -1)  # Зсуваємо лаги
    last_known[0, -1] = next_prediction  # Додаємо прогнозоване значення

# 12. Відображення прогнозів у майбутнє
print("Future Predictions:", future_predictions)

plt.figure(figsize=(10, 6))
plt.plot(range(len(data)), data['Close'], label='Historical Data', color='blue')
plt.show()
plt.plot(range(len(future_steps)), future_predictions, label='Future Predictions', color='red')
plt.legend()
plt.title('SVR Future Predictions')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(data))[-100:], data['Close'][-100:], label='Historical Data', color='blue')
plt.plot(range(len(data)[-100:], (len(data) + future_steps)[-100:]), future_predictions[-100:], label='Future Predictions', color='red')
plt.legend()
plt.title('SVR Future Predictions')
plt.show()
